### Tutorial on Space-time FEM with FEniCSx
Dominik Kern ORCID [0000-0002-1958-2982](https://orcid.org/0000-0002-1958-2982) 

This notebook is a supplement to the tutorial TODO zenodo.doi

**solving the non-dimensional heat equation in a 1D bar using a full space-time finite element method** 

In [ ]:
import numpy as np
import ufl
from dolfinx.fem import (Constant, Function, functionspace, dirichletbc, form, assemble_scalar, Expression)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.fem import locate_dofs_geometrical
from dolfinx import plot
from dolfinx import default_scalar_type
from dolfinx import mesh 
from mpi4py import MPI
import matplotlib.pyplot as plt
import pyvista as pv

#### parameters

In [ ]:
comm = MPI.COMM_WORLD
nx = 4 # Number of spatial elements
nt = 8 # Number of temporal elements
order = 1  # Polynomial order

#### discretization

In [ ]:
domain = mesh.create_unit_square(comm, nx, nt, cell_type=mesh.CellType.quadrilateral) #

V = functionspace(domain, ("Lagrange", order))


# Define the initial, i.e. time boundary, condition u(x, 0)
def initial_condition_func(x):
    return np.sin(np.pi * x[0])

u_initial = Function(V)
u_initial.interpolate(initial_condition_func)
initial_dofs = locate_dofs_geometrical(V, lambda x: np.isclose(x[1], 0))
bc_initial = dirichletbc(u_initial, initial_dofs)

# Define the spatial boundary condition u(0, t)
left_boundary_dofs = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0))
bc_left = dirichletbc(np.float64(0.0), left_boundary_dofs, V)

# Define the spatial boundary condition u(1, t) 
right_boundary_dofs = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 1))
bc_right = dirichletbc(np.float64(0.0), right_boundary_dofs, V)

# Combine all boundary conditions (space and time)
bcs = [bc_initial, bc_left, bc_right]

# ---   3. Define the variational problem   ---
u = ufl.TrialFunction(V)
Du = ufl.TestFunction(V)

# Weak form
a = (ufl.grad(u)[1] * Du + ufl.grad(u)[0] * ufl.grad(Du)[0]) * ufl.dx
L = Constant(domain, np.float64(0.0)) * Du * ufl.dx # Homogeneous PDE

##### solution

In [ ]:
# This solves for the entire space-time solution at once.
problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.name = "u_spacetime"

#### post-processing

In [ ]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)
u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_warped = u_grid.warp_by_scalar()
plotter = pv.Plotter()
plotter.add_mesh(u_warped, show_edges=True, scalar_bar_args={'vertical':True})
if not pv.OFF_SCREEN:
    plotter.show_grid(xlabel="x", ylabel="t", zlabel="u")
    plotter.show()